In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
import string
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv('TiktokShop.csv', delimiter=';')

In [4]:
print('Ukuran Dataset: ', df.shape)
print('Kolom: ', df.columns)

Ukuran Dataset:  (55, 12)
Kolom:  Index(['created_at', 'id_str', 'full_text', 'quote_count', 'reply_count',
       'retweet_count', 'favorite_count', 'lang', 'user_id_str',
       'conversation_id_str', 'username', 'tweet_url'],
      dtype='object')


In [5]:
df.head(5)

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
0,Tue Sep 26 21:43:14 +0000 2023,1706786532680777750,@afvckado_ Padahal tiktokshop tuh enak buat be...,0,4,0,0,in,1582554721,1706638181410726255,princeofmean,https://twitter.com/princeofmean/status/170678...
1,Tue Sep 26 16:55:27 +0000 2023,1706714109558112734,#TikTokShopDitutup #TikTokShopDilarang #Transf...,0,0,0,0,qht,1628672434881851393,1706714095263809622,kolega_ganjar,https://twitter.com/kolega_ganjar/status/17067...
2,Tue Sep 26 16:27:26 +0000 2023,1706707059532759161,Setelah 2 bulan trial n error ngurus tiktoksho...,0,1,0,0,in,1359078697559445505,1706707059532759161,sereneerorra,https://twitter.com/sereneerorra/status/170670...
3,Tue Sep 26 15:25:18 +0000 2023,1706691423083639140,Pemerintah perlu edukasi dan fasilitasi masyar...,0,0,0,0,in,1278555760520163333,1706691415462539431,privacyjigeum1_,https://twitter.com/privacyjigeum1_/status/170...
4,Tue Sep 26 14:28:21 +0000 2023,1706677090878431300,Klo tiktokshop udah dilarang tapi masih sepi j...,0,0,0,0,in,234276644,1706677090878431300,klmnopqruss,https://twitter.com/klmnopqruss/status/1706677...


In [6]:
# Menghapus kolom dan menyisakan kolom created_at. full_text, lang dan user_id_str
kolom_hapus = ['id_str','quote_count','reply_count','retweet_count','favorite_count','lang','user_id_str','conversation_id_str','tweet_url']
df = df.drop(kolom_hapus, axis=1)

df.head(5)

,created_at,full_text,username
0,Tue Sep 26 21:43:14 +0000 2023,@afvckado_ Padahal tiktokshop tuh enak buat be...,princeofmean
1,Tue Sep 26 16:55:27 +0000 2023,#TikTokShopDitutup #TikTokShopDilarang #Transf...,kolega_ganjar
2,Tue Sep 26 16:27:26 +0000 2023,Setelah 2 bulan trial n error ngurus tiktoksho...,sereneerorra
3,Tue Sep 26 15:25:18 +0000 2023,Pemerintah perlu edukasi dan fasilitasi masyar...,privacyjigeum1_
4,Tue Sep 26 14:28:21 +0000 2023,Klo tiktokshop udah dilarang tapi masih sepi j...,klmnopqruss


In [7]:
df['full_text'] = df['full_text'].str.lower()

In [8]:
df.head(5)

,created_at,full_text,username
0,Tue Sep 26 21:43:14 +0000 2023,@afvckado_ padahal tiktokshop tuh enak buat be...,princeofmean
1,Tue Sep 26 16:55:27 +0000 2023,#tiktokshopditutup #tiktokshopdilarang #transf...,kolega_ganjar
2,Tue Sep 26 16:27:26 +0000 2023,setelah 2 bulan trial n error ngurus tiktoksho...,sereneerorra
3,Tue Sep 26 15:25:18 +0000 2023,pemerintah perlu edukasi dan fasilitasi masyar...,privacyjigeum1_
4,Tue Sep 26 14:28:21 +0000 2023,klo tiktokshop udah dilarang tapi masih sepi j...,klmnopqruss


In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
def remove_punct(text):
    #Menghilangkan URL
    text = re.sub(r'http\S+|https\S+','',text,flags=re.MULTILINE)
    #Mengilangkan @mention
    text = re.sub(r'@\w+','',text)
    #Menghilangkan #hastag
    text = re.sub(r'#\w+','',text)
    #Menghilangkan #angka
    text = re.sub(r'\d+','',text)
    #Menghilangkan tanda baca
    text = re.sub(r'[^\w\s]','',str (text))
    text = re.sub(r'\b\w{1,2}\b]','',text) #Menghilangkan kata dengan panjang 1 atau 2 karakter
    text = re.sub(r'\s\s+','',text)
    return text

df['preprocessing'] = df['full_text'].apply(lambda x: remove_punct(x))

In [11]:
df.head(5)

,created_at,full_text,username,preprocessing
0,Tue Sep 26 21:43:14 +0000 2023,@afvckado_ padahal tiktokshop tuh enak buat be...,princeofmean,padahal tiktokshop tuh enak buat belanja samb...
1,Tue Sep 26 16:55:27 +0000 2023,#tiktokshopditutup #tiktokshopdilarang #transf...,kolega_ganjar,
2,Tue Sep 26 16:27:26 +0000 2023,setelah 2 bulan trial n error ngurus tiktoksho...,sereneerorra,setelahbulan trial n error ngurus tiktokshop d...
3,Tue Sep 26 15:25:18 +0000 2023,pemerintah perlu edukasi dan fasilitasi masyar...,privacyjigeum1_,pemerintah perlu edukasi dan fasilitasi masyar...
4,Tue Sep 26 14:28:21 +0000 2023,klo tiktokshop udah dilarang tapi masih sepi j...,klmnopqruss,klo tiktokshop udah dilarang tapi masih sepi j...


In [12]:
import nltk
from nltk.tokenize import word_tokenize

# Tokenisasi teks pada kolom tertentu (misalnya 'teks') dan tambahkan hasil token ke kolom baru ('token')
df['preprocessing'] = df['preprocessing'].apply(lambda x: word_tokenize(x.lower()))

In [13]:
df.head(5)

,created_at,full_text,username,preprocessing
0,Tue Sep 26 21:43:14 +0000 2023,@afvckado_ padahal tiktokshop tuh enak buat be...,princeofmean,"[padahal, tiktokshop, tuh, enak, buat, belanja..."
1,Tue Sep 26 16:55:27 +0000 2023,#tiktokshopditutup #tiktokshopdilarang #transf...,kolega_ganjar,[]
2,Tue Sep 26 16:27:26 +0000 2023,setelah 2 bulan trial n error ngurus tiktoksho...,sereneerorra,"[setelahbulan, trial, n, error, ngurus, tiktok..."
3,Tue Sep 26 15:25:18 +0000 2023,pemerintah perlu edukasi dan fasilitasi masyar...,privacyjigeum1_,"[pemerintah, perlu, edukasi, dan, fasilitasi, ..."
4,Tue Sep 26 14:28:21 +0000 2023,klo tiktokshop udah dilarang tapi masih sepi j...,klmnopqruss,"[klo, tiktokshop, udah, dilarang, tapi, masih,..."


In [14]:
#stopword removal kata tidak penting
stopword = nltk.corpus.stopwords.words('indonesian')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
df['preprocessing'] = df['preprocessing'].apply(lambda x: remove_stopwords(x))

In [15]:
df.head(5)

,created_at,full_text,username,preprocessing
0,Tue Sep 26 21:43:14 +0000 2023,@afvckado_ padahal tiktokshop tuh enak buat be...,princeofmean,"[tiktokshop, tuh, enak, belanja, nontonin, liv..."
1,Tue Sep 26 16:55:27 +0000 2023,#tiktokshopditutup #tiktokshopdilarang #transf...,kolega_ganjar,[]
2,Tue Sep 26 16:27:26 +0000 2023,setelah 2 bulan trial n error ngurus tiktoksho...,sereneerorra,"[setelahbulan, trial, n, error, ngurus, tiktok..."
3,Tue Sep 26 15:25:18 +0000 2023,pemerintah perlu edukasi dan fasilitasi masyar...,privacyjigeum1_,"[pemerintah, edukasi, fasilitasi, masyarakat, ..."
4,Tue Sep 26 14:28:21 +0000 2023,klo tiktokshop udah dilarang tapi masih sepi j...,klmnopqruss,"[klo, tiktokshop, udah, dilarang, sepi, gimana..."


In [16]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming_text(text):
    # Inisialisasi Stemmer
    stemmer = StemmerFactory().create_stemmer()
    # Menerapkan stemming pada setiap kata dalam teks
    stemmed_text = [stemmer.stem(word) for word in text]
    return stemmed_text

df['preprocessing'] = df['preprocessing'].apply(lambda x: stemming_text(x) if isinstance(x, list) else x)

In [17]:
# Fungsi untuk mengembalikan proses tokenisasi ke text awal
def join_tokens(tokens):
    return' '.join(tokens)

df['preprocessing'] = df['preprocessing'].apply(join_tokens)

In [18]:
print(df['preprocessing'].head())  # Cetak beberapa baris pertama sebelum penggabungan
df['preprocessing'] = df['preprocessing'].apply(join_tokens)
print(df['preprocessing'].head())  # Cetak beberapa baris pertama setelah penggabungan

0    tiktokshop tuh enak belanja nontonin live nyab...
1                                                     
2    setelahbulan trial n error ngurus tiktokshop a...
3    perintah edukasi fasilitas masyarakat ga mesti...
4    klo tiktokshop udah larang sepi gimana tuh nya...
Name: preprocessing, dtype: object
0    t i k t o k s h o p   t u h   e n a k   b e l ...
1                                                     
2    s e t e l a h b u l a n   t r i a l   n   e r ...
3    p e r i n t a h   e d u k a s i   f a s i l i ...
4    k l o   t i k t o k s h o p   u d a h   l a r ...
Name: preprocessing, dtype: object


In [19]:
df.to_csv('pra-pemrosesan.csv', index=False)
print('Data berhasil disimpan ke dalam file CSV.')

Data berhasil disimpan ke dalam file CSV.
